# Extract, Transform, Load

Before you can run this notebook, make sure you have Python 3.10 installed and execute `python3 -m pip install -r requirements.txt`.

In [ ]:
%reload_ext autoreload
%autoreload 2
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import wavescapes
from wavescapes.color import circular_hue

import etl
import utils

Configuring the notebook to produce the defaults from the paper. For more information on available normalization methods see the section "Loading magnitude-phase matrices" below.

In [ ]:
DEBUSSY_REPO = '..'
DATA_FOLDER = '.'
DEFAULT_FIGURE_SIZE = 2286
EXAMPLE_FNAME = 'l123-08_preludes_ondine'
how = '0c'
indulge = True
norm_method = (how, indulge)

## Loading metadata
Metadata for all pieces contained in the dataset.

In [ ]:
metadata = etl.get_metadata(DEBUSSY_REPO)
metadata.head()

The column `year` contains composition years as the middle between beginning and end  of the composition span.

In [ ]:
metadata.year.head(10)

Series `median_recording` contains median recording times in seconds, retrieved from the Spotify API. the Spotify API.

In [ ]:
metadata.median_recording.head(10)

Columns mirroring a piece's activity are currently:
* `qb_per_minute`: the pieces' lengths (expressed as 'qb' = quarterbeats) normalized by the median recording times; a proxy for the tempo
* `sounding_notes_per_minute`: the summed length of all notes normalized by the piece's duration (in minutes)
* `sounding_notes_per_qb`: the summed length of all notes normalized by the piece's length (in qb)
Other measures of activity could be, for example, 'onsets per beat/second' or 'distinct pitch classes per beat/second'.

## Loading Pitch Class Vectors (PCVs)
An `{fname -> pd.DataFrame}` dictionary where each `(NX12)` DataFrame contains the absolute durations (expressed in quarter nots) of the 12 chromatic pitch classes for the `N` slices of length = 1 quarter note that make up the piece `fname`. The IntervalIndex reflects each slice's position in the piece. Set `pandas` to False to retrieve NumPy arrays without the IntervalIndex and column names.

In [ ]:
pcvs = etl.get_pcvs(DEBUSSY_REPO, pandas=True)
etl.test_dict_keys(pcvs, metadata)
pcvs[EXAMPLE_FNAME].head(5)

The `wavescapes` library allows for creating a wavescape directly from a PCV matrix. Here es one showing the second coefficient: 

In [ ]:
coeff = 2
label = etl.make_wavescape_label(EXAMPLE_FNAME, how, indulge, coeff=coeff)
path = os.path.join('figures', etl.make_filename(EXAMPLE_FNAME, how, indulge, coeff=coeff, ext='.png'))
wavescapes.single_wavescape_from_pcvs(pcvs[EXAMPLE_FNAME],
                                      width=DEFAULT_FIGURE_SIZE,
                                      coefficient=coeff,
                                      save_label=path,
                                      magn_stra=how,
                                      output_rgba=True,
                                      aw_per_tick=10,
                                      tick_factor=10,
                                      label=label,
                                      label_size=20)

## Loading Pitch Class Matrices
An `{fname -> np.array}` dictionary where each `(NxNx12)` array contains the aggregated PCVs for all segments that make up a piece. The square matrices contain values only in the upper right triangle, with the lower left beneath the diagonal is filled with zeros. The values are arranged such that row 0 correponds to the original PCV, row 1 the aggregated PCVs for all segments of length = 2 quarter notes, etc. For getting the segment reaching from slice 3 to 5 (including), i.e. length 3, the coordinates are `(2, 5)` (think x = 'length - 1' and y = index of the last slice included).

The following example shows the upper left 3x3 submatrix where
* the first three entries (which are PCVs of size 12) correspond to the pitch class distributions of the piece's first three quarternote slices,
* the two last vectors of the second row each correspond to a sum of two adjacent vectors above, and
* the last entry of the the third row corresponds to the sum all three PCVs.

In [ ]:
pcms = etl.get_pcms(DEBUSSY_REPO)
etl.test_dict_keys(pcms, metadata)
print(f"Shape of the PCM for {EXAMPLE_FNAME}: {pcms[EXAMPLE_FNAME].shape}")
pcms[EXAMPLE_FNAME][:3, :3]

## Loading Discrete Fourier Transforms
`{fname -> np.array}` containing `(NxNx7)` complex matrices. For instance, here's the first element, a size 7 complex vector with DFT coefficients 0 through 6:

In [ ]:
dfts = etl.get_dfts(DEBUSSY_REPO)
etl.test_dict_keys(dfts, metadata)
print(f"Shape of the DFT for {EXAMPLE_FNAME}: {dfts[EXAMPLE_FNAME].shape}")
dfts[EXAMPLE_FNAME][0,0]

You can view the 7 complex numbers as magnitude-phase pairs. In the following we use magnitude-phase-matrices of this format.

In [ ]:
utils.get_coeff(dfts[EXAMPLE_FNAME], 0, 0, deg=True)

For convenience, values can also be inspected as strings where the numbers are rounded and angles are shown in degrees:

In [ ]:
utils.get_coeff(dfts[EXAMPLE_FNAME], 0, 0, deg=True)

## Loading magnitude-phase matrices
`{fname -> np.array}` where each of the `(NxNx6x2)` matrices contains the 6 relevant DFT coefficients converted into magnitude-phase pairs where the magnitudes have undergone at least one normalization, i.e. are all within [0,1]. The first time the notebook runs, the matrices are computed and pickled to disk, from where they can be loaded on later runs.

The parameter `norm_params` can be one or several `(how, indulge)` pairs where `indulge` is a boolean and `how ∈ {'0c', 'post_norm', 'max_weighted', 'max'}`.

### Normalizing magnitudes

The available normalization methods for `how` are:
* **'0c'** default normalisation, will normalise each magnitude by the 0th coefficient (which corresponds to the sum of the weight of each pitch class). This ensures onlypitch class distribution whose periodicity exactly match the coefficient's periodicity can reach the value of 1.
* **'post_norm'** based on the 0c normalisation but "boost" the space of all normalized magnitude so the maximum magnitude observable is set to the max opacity value. This means that if any PCV in the utm given as input reaches the 0c normalized magnitude of 1, this parameter acts like the '0c' one. This magn_strat should be used with audio input mainly, as seldom PCV derived from audio data can reach the maximal value of normalized magnitude for any coefficient.
* **'max'** set the grayscal value 1 to the maximum possible magnitude in the wavescape, and interpolate linearly all other values of magnitude based on that maximum value set to 1. Warning: will bias the visual representation in a way that the top of the visualisation will display much more magnitude than lower levels.
* **'max_weighted'** same principle as max, except the maximum magnitude is now taken at the hierarchical level, in other words, each level will have a different opacity mapping, with the value 1 set to the maximum magnitude t this level. This normalisation is an attempt to remove the bias toward higher hierarchical level that is introduced by the 'max' magnitude process cited previously.

`indulge` is an additional normalization that we apply to the magnitude based on the phase. Since magnitudes of 1 are possible only for a prototypical phase sitting on the unit circle, you can set this parameter to True to normalize the magnitudes by the maximally achievable magnitude given the phase which is bounded by straight lines between adjacent prototypes. (Musical prototypes are visualized in the [midiVERTO webApp](https://dcmlab.github.io/midiVERTO/#/analysis)) The pitch class vectors that benefit most from this normalization in terms of magnitude gain are those whose phase is exactly between two prototypes, such as the "octatonic" combination O₀,₁. The maximal "boosting" factors for the first 5 coefficients are `{1: 1.035276, 2: 1.15470, 3: 1.30656, 4: 2.0, 5: 1.035276}`. The sixth coefficient's phase can only be 0 or pi so it remains unchanged. Use this option if you want to compensate for the smaller magnitude space of the middle coefficients.

In [ ]:
mag_phase_path = os.path.join(DATA_FOLDER, 'pickled_magnitude_phase_matrices')
mag_phase_mx_dict = etl.get_magnitude_phase_matrices(dfts=dfts, data_folder=mag_phase_path, norm_params=norm_method)
etl.test_dict_keys(mag_phase_mx_dict, metadata)
print(f"Shape of the magnitude-phase matrix for {EXAMPLE_FNAME}: {mag_phase_mx_dict[EXAMPLE_FNAME].shape}")

## Summary wavescapes

This cell depends on the previously loaded magnitude-phase matrices, i.e. a conscious choice of a normalization method has been made above.

`get_most_resonant` returns three `{fname -> nd.array}` dictionaries where for each piece, the three `(NxN)` matrices correspond to

1. the index between 0 and 5 of the most resonant of the six DFT coefficient 1 through 6
2. its magnitude
3. the inverse entropy of the 6 magnitudes

The following example shows these 3 values for the bottom row of the example summary wavescape.

In [ ]:
max_coeffs, max_mags, inv_entropies = etl.get_most_resonant(mag_phase_mx_dict)
np.column_stack((max_coeffs[EXAMPLE_FNAME][:3],
max_mags[EXAMPLE_FNAME][:3],
inv_entropies[EXAMPLE_FNAME][:3]))

In order to plot these values in a wavescape, they need to be transformed into color values. The function `most_resonant2color()` attributes six equidistant hues to the most resonant coefficients and takes one of the other two matrices to adapt the opacity.

In the first example the opacity shows the magnitude of the most resonant coefficient:

In [ ]:
colors = utils.most_resonant2color(max_coeffs[EXAMPLE_FNAME], max_mags[EXAMPLE_FNAME])
by_entropy = False
label = etl.make_wavescape_label(EXAMPLE_FNAME, how, indulge, by_entropy=by_entropy)
ws = wavescapes.Wavescape(colors, width=DEFAULT_FIGURE_SIZE)
ws.draw(aw_per_tick=10, tick_factor=10, label=label, label_size=20)
path = os.path.join('figures', etl.make_filename(EXAMPLE_FNAME, how, indulge, summary_by_entropy=by_entropy, ext='.png'))
plt.savefig(path)

In the second example, the opacity corresponds to the inverse normalized entropy of the 6 magnitudes. In other words, opacity is maximal if the most resonant coefficient is the only one with magnitude > 0; whereas the color is white when all coefficients have the same magnitude.

In [ ]:
colors = utils.most_resonant2color(max_coeffs[EXAMPLE_FNAME], inv_entropies[EXAMPLE_FNAME])
by_entropy = True
label = etl.make_wavescape_label(EXAMPLE_FNAME, how, indulge, by_entropy=by_entropy)
ws = wavescapes.Wavescape(colors, width=DEFAULT_FIGURE_SIZE)
ws.draw(aw_per_tick=10, tick_factor=10, label=label, label_size=20)
path = os.path.join('figures', etl.make_filename(EXAMPLE_FNAME, how, indulge, summary_by_entropy=by_entropy, ext='.png'))
plt.savefig(path)

# Metrics

In this section, we compile a dataframe containing all metrics results, both melted and not. The metrics will then be stored and used for testing in RFILE. Optional plots and tests can be done by adjusting the parameters of the wrapper function `get_metric` that can be found in `etl.py`. 

In [ ]:
# let's start from the available metadata for each piece
metadata_metrics = metadata.copy()

# Prevalence of each coefficient

Prevalence of coefficient $n$ in a piece: $W(n)=1/N \sum_{i \in S(n)} i$ where $N$ is the total number of nodes in the wavescape, $S(n)$ is the set of the indices of the nodes in the summary wavescapes that are attributed to coefficient $n$ (i.e., where coefficient $n$ is the most prominent among the six).

In [ ]:
# defining column names
cols = [f"percentage_resonances_{i}" for i in range(1,7)]

metadata_metrics = etl.get_metric('percentage_resonance', metadata_metrics, 
                              max_coeffs=max_coeffs, cols=cols, 
                              store_matrix=True, show_plot=True, unified=True,
                              save_name='percentage_resonance', title='Percentage Resonance')

In [ ]:
metadata_metrics[cols].head()

In order to account for the certainty that a certain coefficient is actually the most resonance, we weigh the previous metric by entropy as follows: $W(n)=1/N \sum_{i \in S(n)} w_i$ where $N$ is the total number of nodes in the wavescape, $S(n)$ is the set of the indices of the nodes in the summary wavescapes that are attributed to coefficient $n$ (i.e., where coefficient $n$ is the most prominent among the six), and $w_i$ is the weight (opacity) of the $i$-th node in the summary wavescape, in this case, the entropy of $i$.



In [ ]:
cols = [f"percentage_resonances_entropy_{i}" for i in range(1,7)]

metadata_metrics = etl.get_metric('percentage_resonance_entropy', metadata_metrics, 
                              cols=cols,
                              max_coeffs=max_coeffs, inv_entropies=inv_entropies,
                              store_matrix=True, show_plot=True, unified=True,
                              save_name='percentage_resonance_entropy', title='Percentage Resonance (entropy)')

In [ ]:
metadata_metrics[cols].head()

# Post-hoc analysis of hierarchical prevalence
## Moment of Inertia

Moment of inertia of coefficient $n$ in the summary wavescape: $I(n)=1/N \sum_{i \in S(n)} w_i y_i^2$, where N is the total number of nodes in the wavescape, $S(n)$ is the set of the indices of the nodes in the summary wavescapes that are attributed to coefficient $n$ (i.e., where coefficient n is the most prominent among the six), $w_i$ is the weight (opacity) of the $i$-th node in the summary wavescape, and $y_i$ is the vertical coordinate of the $i$-th node in the summary wavescape


In [ ]:
cols = [f"moments_of_inertia_{i}" for i in range(1,7)]

metadata_metrics = etl.get_metric('moment_of_inertia', metadata_metrics, 
                              cols=cols,
                              max_coeffs=max_coeffs, max_mags=max_mags,
                              store_matrix=True, show_plot=True, unified=True,
                              save_name='moments_of_inertia', title='Moments of Inertia')

In [ ]:
metadata_metrics[cols].head()

# Measure Theoretic Entropy

Measure-theoretic entropy: Let $A={A_1,...,A_k}$ be a (finite) partition of a probability space $(X,P(X),)$: the entropy of the partition $A$ is defined as $H(A)= - \sum_{i} \mu(A_i) \log \mu(A_i)$. We can take $X$ as the support of the wavescape, $A$ as the set of the connected regions in the unified wavescape, and $\mu(Y)=(area-of-Y)/(area-of-X)$ for any subset $Y$ of the wavescape.


In [ ]:
cols = 'partition_entropy'

metadata_metrics = etl.get_metric('partition_entropy', metadata_metrics, 
                              cols=cols,
                              max_coeffs=max_coeffs,
                              store_matrix=True, scatter=True, show_plot=True, unified=True, 
                              save_name='partition_entropy', title='Partition Entropy')

In [ ]:
metadata_metrics[cols].head()

# Decreasing magnitude in height

The inverse coherence is the slope of the regression line that starts from the magnitude resonance in the summary wavescape at bottom of the wavescape and reaches the one at the top of the wavescape.

In [ ]:
cols = 'inverse_coherence'
metadata_metrics = etl.get_metric('inverse_coherence', metadata_metrics, 
                              cols=cols,
                              max_mags=max_mags,
                              store_matrix=True, show_plot=True, unified=True, scatter=True,
                              save_name='inverse_coherence', title='Inverse Coherence')

In [ ]:
metadata_metrics[cols].head()

# Storing the final metrics files

In [ ]:
if not os.path.isdir('results'):
    os.makedirs('results')

metadata_metrics.reset_index().to_csv('results/results.csv', index=False)

In [ ]:
# melting the results to be used for testing on coefficient specific metrics

cols = [col for col in metadata_metrics.columns if col[-1].isnumeric()]
metadata_metrics = metadata_metrics.reset_index()
metadata_melted = pd.melt(metadata_metrics, id_vars=['fname', 'length_qb', 'year', 'last_mc'], value_vars=cols)

metadata_melted.reset_index().to_csv('results/results_melted.csv', index=False)